<h1><center>Regressão Logística</center></h1>

<h2>Introdução</h2>

Esse notebook é uma junção de exercícios de regressões logísticas do curso <a href='https://www.udemy.com/the-data-science-course-complete-data-science-bootcamp/'>The Data Science Course 2019: The Complete Data Science Bootcamp</a>, onde cada exercício iria complementando e inserindo novos parâmetros para a análise. Para esse exercício nos é dado um dataset da campanha de marketing da instituição bancária portuguesa.<br>
Aproveitando este exercício, vou utilizar estes dados para dar continuidade no meu caderno de estudo no curso, que servirá para mim posteriormente como fonte de revisão.

<a name='INDICE'></a>
<h2>Índice</h2>

<ol>
    <li><a href='#REG_LOG'>Regressão Logística</a></li>
    <li><a href='#EXERCICIO'>Exercício</a></li>
    <ol>
        <li><a href='#BIBLIOTECA'>Importando bibliotecas</a></li>
        <li><a href='#EXPLORAR'>Explorando os dados</a></li>
        <li><a href='#REGRESSAO'>Criando a regressão</a></li>
        <li><a href='#ANALISE'>Analisando o modelo</a></li>
        <li><a href='#TESTE'>Testando o modelo</a></li>
        <li><a href='#VALIDANDO'>Validando o modelo</a></li>
    </ol>
</ol>

<a name='REG_LOG'></a>
<h2>Regressão Logística</h2>

Nem sempre é possível utilizar a Regressão Linear, dependendo da sua variável dependente seja necessário realizar uma transformação para que as 5 suposições da regressão linear não sejam violadas ou até utilizar outro tipo de regressão. Um exemplo disto é quando a variável dependente(alvo) é categórica e para este caso é necessário utilizar a Regressão Logística. Assim como a regressão linear a regressão logística utiliza os dados que fornecemos para compreender o comportamento do conjuto e por assim fazermos uma análise preditiva.<br>
<img src='./img/reg_comparisson.jpeg'/>
Como o nome diz a regressão utiliza o modelo logístico que calcula a probabilidade de um evento ocorrer retornando valores de 0 a 1 e o seu modelo de regressão logística é descrito da seguinte forma:

\begin{align}
\ p(x) = \frac{e^{(\beta_{0} + \beta_{1}x_{1} + \cdots + \beta_{k}x_{k})}}{1 + e^{(\beta_{0} + \beta_{1}x_{1} + \cdots + \beta_{k}x_{k})}}
\end{align}
<br>

Porém é uma fórmula muito complicada, por isso utilizamos o Modelo Linear Generalizado (GLM) do modelo de regressão logística, que é modelo de regressão logit, que é descrito da seguinte forma:

\begin{align}
\frac{p(x)}{1 - p(x)} = e^{(\beta_{0} + \beta_{1}x_{1} + \cdots + \beta_{k}x_{k})}
\end{align}
<br>

Comparando as duas já é possível ver o motivo de usarem a segunda fórmula, ela é muito mais intuitiva, fora que a primeira parte da equação é conhecido como probabilidade(odds).

<a href='#INDICE'>Voltar para o índice</a><br>

<a name='EXERCICIO'></a>
<h2>Exercício</h2>

Após toda essa teoria, chegou a hora de colocarmos em prática. Para resolver esse exercício iremos utilizar a biblioteca StatsModel, então será necessário importar algumas bibliotecas.<br>

<a name='BIBLIOTECA'></a>
<h3>Importando as bibliotecas</h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline

In [2]:
raw_df = pd.read_csv('Bank-data.csv')
raw_df.head()

,Unnamed: 0,interest_rate,credit,march,may,previous,duration,y
0,0,1.334,0.0,1.0,0.0,0.0,117.0,no
1,1,0.767,0.0,0.0,2.0,1.0,274.0,yes
2,2,4.858,0.0,1.0,0.0,0.0,167.0,no
3,3,4.120,0.0,0.0,0.0,0.0,686.0,yes
4,4,4.856,0.0,1.0,0.0,0.0,157.0,no


Como vamos alterar alguns valores no dataframe, vamos realizar uma cópia deste dataframe para outra variável.

In [3]:
df = raw_df.copy()

In [4]:
df.drop(['Unnamed: 0'], axis = 1, inplace=True)
df['y'] = df['y'].map({'yes':1, 'no':0})
df.head()

,interest_rate,credit,march,may,previous,duration,y
0,1.334,0.0,1.0,0.0,0.0,117.0,0
1,0.767,0.0,0.0,2.0,1.0,274.0,1
2,4.858,0.0,1.0,0.0,0.0,167.0,0
3,4.120,0.0,0.0,0.0,0.0,686.0,1
4,4.856,0.0,1.0,0.0,0.0,157.0,0


Como não há nenhum dado faltante, podemos começar a explorar o nosso dataset.

<a name='EXPLORAR'></a>
<h3>Explorando os dados</h3>

Primeiro vamos utilizar do método <i>describe</i> para vermos a tendência central e a dispersão dos nossos dados.

In [5]:
df.describe()

,interest_rate,credit,march,may,previous,duration,y
count,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000
mean,2.835776,0.034749,0.266409,0.388031,0.127413,382.177606,0.500000
std,1.876903,0.183321,0.442508,0.814527,0.333758,344.295990,0.500483
min,0.635000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000
25%,1.042750,0.000000,0.000000,0.000000,0.000000,155.000000,0.000000
50%,1.466000,0.000000,0.000000,0.000000,0.000000,266.500000,0.500000
75%,4.956500,0.000000,1.000000,0.000000,0.000000,482.750000,1.000000
max,4.970000,1.000000,1.000000,5.000000,1.000000,2653.000000,1.000000


<a name='REGRESSAO'></a>
<h3>Criando a Regressão</h3>

Antes de criarmos a regressão precisamos separar qual é a nossa variável dependente e independente.

In [6]:
x = df[['interest_rate','credit','march','previous','duration']]
y = df['y']

In [7]:
reg_logit = sm.Logit(y,x)
results_logit = reg_logit.fit()
results_logit.summary()

Optimization terminated successfully.
         Current function value: 0.336668
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  518
Model:                          Logit   Df Residuals:                      513
Method:                           MLE   Df Model:                            4
Date:                Wed, 22 May 2019   Pseudo R-squ.:                  0.5143
Time:                        17:21:30   Log-Likelihood:                -174.39
converged:                       True   LL-Null:                       -359.05
                                        LLR p-value:                 1.185e-78
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
interest_rate    -0.8030      0.079    -10.201      0.000      -0.957      -0.649
credit            2.3459      1.071      2.190      0.029       0.246       4.445
march            -1.8387      0.315     -5.831      0.000      -2.457      -1.221
previous          1.5262      0.478      3.190      0.001       0.588       2.464
duration          0.0069      0.001     10.365      0.000       0.006       0.008
=================================================================================
"""

<a name='ANALISE'></a>
<h3>Analisando o modelo</h3>

Após criarmos o modelo, devemos analisá-lo, para isso o método <i>summary</i> do <i>StatsModels</i> nos dá uma série de parâmetros que podemos utilizar para a nossa análise. Primeiro verificamos o p-value das variáveis, para verificar se todos nossos estimadores são significantes para a regressão, e felizmente todos os p-value são aproximadamente 0.000, excluindo a variável <i>credit</i> porém pelo valor do p-value continua sendo significante.<br>
Após isso verificamos os seguintes parâmetros:
<ul>
    <li><b>MLE:</b> é um método que obtém a estimativa do parâmetro encontrando o valor que maximiza a função de verossimilhança (likelihood function).</li>
    <li><b>Likelihood Function:</b> obtém a probabilidade relativa de obter o valor dado os conjutos passados.</li>
    <li><b>Log-likelihood:</b> é o logarítmo da função de verossimilhança, é o mesmo que a estimativa por máxima verossimilhança, pois a função logarítma é restritamente crescente. Normalmente este parâmetro é negativo e quanto maior melhor.</li>
    <li><b>Log-likelihood Null (LL-Null):</b> é o valor de do log-likelihood de um modelo que não tem variável independente, ou seja, um modelo que não tem explicabilidade.</li>
    <li><b>Log-likelihood Ratio (LLR p-value):</b> teste de hipótese que compara com o LL-Null para verificar a significância do modelo.</li>
    <li><b>Pseudo R-squared:</b> é o <i>McFadden's R-squared</i> e de acordo com o criador um bom valor está entre 0.2 e 0.4.</li>
</ul>

<a name='TESTE'></a>
<h3>Testando o modelo</h3>

Com o modelo treinado, vamos testá-lo. Para isso vamos usar o comando <i>pred_table</i>, que nos devolve uma matriz de confusão. A matriz de confusão nos descreve a performance do modelo em relação a classificação e é usada para calcular o <i>Recall</i>, <i>Precisão</i>, <i>Acurácia</i> e <i>F-score</i>.

In [8]:
conf_matrx = pd.DataFrame(results_logit.pred_table())
conf_matrx.index = ['Real: Positivo', 'Real: Negativo']
conf_matrx.columns = ['Previsto: Positivo', 'Previsto: Negativo']
conf_matrx

,Previsto: Positivo,Previsto: Negativo
Real: Positivo,218.0,41.0
Real: Negativo,28.0,231.0


Agora com a matriz de confusão ilustrada, é possível ver que temos 4 campos e eles são descritos da seguinte forma:
<ul>
    <li><b>Verdadeiro Positivo (TP):</b> é o campo no valor de <i>218</i> e ele diz quantas vezes a regressão previu como positivo e o valor real era positivo.</li>
    <li><b>Falso Positivo (FP) - Erro Tipo 1:</b> é o campo no valor de <i>28</i> e ele diz quantas vezes a regressão previu como positivo e o valor real era negativo.</li>
    <li><b>Verdadeiro Negativo (TN):</b> é o campo no valor de <i>231</i> e ele diz quantas vezes a regressão previu como negativo e o valor real era negativo</li>
    <li><b>Falso Negativo (FN) - Erro Tipo 2:</b> é o campo no valor de <i>41</i> e ele diz quantas vezes a regressão previu como negativo e o valor real era positivo.</li>
</ul>

Como isso em mente podemos calcular os parâmetros <i>Recall</i>, <i>Precisão</i>, <i>Acurácia</i> e <i>F-score</i>.

<h4>Recall</h4>
Mensura de todos os valores positivos quantos o modelo previu corretamente.<br><br>
\begin{align}
\ recall = \frac{TP}{TP + FN}
\end{align}

<h4>Precisão</h4>
Mensura de todos os positivos previstos quantos foram previstos corretamente.<br><br>
\begin{align}
\ precisão = \frac{TP}{TP + FP}
\end{align}

<h4>Acurácia</h4>
Mensura de todos os valores previstos quantos foram previstos corretamente.<br><br>
\begin{align}
\ acurácia = \frac{TP + TN}{TP + FP + TN + FN}
\end{align}

<h4>F-score</h4>
Fornece uma medida mais realista da performance do modelo usando a <i>precisão</i> e <i>recall</i>.<br><br>
\begin{align}
\ F_{1} = 2 * \frac{precisão * recall}{precisão + recall}
\end{align}

In [9]:
recall = conf_matrx.iloc[0,0]/(conf_matrx.iloc[0,0]+conf_matrx.iloc[0,1])
precisao = conf_matrx.iloc[0,0]/(conf_matrx.iloc[0,0]+conf_matrx.iloc[1,0])
acuracia = (conf_matrx.iloc[0,0]+conf_matrx.iloc[1,1])/conf_matrx.sum().sum()
fscore = 2*((precisao*recall)/(precisao+recall))
print('Recall: {0}'.format(round(recall,2)))
print('Precisão: {0}'.format(round(precisao,2)))
print('Acurácia: {0}'.format(round(acuracia,2)))
print('F-score: {0}'.format(round(fscore,2)))

Recall: 0.84
Precisão: 0.89
Acurácia: 0.87
F-score: 0.86


<a name='VALIDANDO'></a>
<h3>Validando o modelo</h3>

Chegamos no última parte do nosso modelo, a validação, faremos igual ao teste porém com uma variável que o nosso modelo nunca viu, simulando como ele se sairia se estivesse em produção. Para isso usamos uma base de dados que já havia sido separada previamente.

In [10]:
df_val = pd.read_csv('Bank-data-testing.csv')
df_val.head()

,Unnamed: 0,interest_rate,credit,march,may,previous,duration,y
0,0,1.313,0.0,1.0,0.0,0.0,487.0,no
1,1,4.961,0.0,0.0,0.0,0.0,132.0,no
2,2,4.856,0.0,1.0,0.0,0.0,92.0,no
3,3,4.120,0.0,0.0,0.0,0.0,1468.0,yes
4,4,4.963,0.0,0.0,0.0,0.0,36.0,no


Como o dataset deve estar exatamente igual ao dataset que o modelo foi treinado, devemos refazer as mesmas alterações que fizemos no dataset de treino.

In [11]:
df_val.drop(['Unnamed: 0'], axis = 1, inplace=True)
df_val['y'] = df_val['y'].map({'yes':1, 'no':0})
df_val.head()

,interest_rate,credit,march,may,previous,duration,y
0,1.313,0.0,1.0,0.0,0.0,487.0,0
1,4.961,0.0,0.0,0.0,0.0,132.0,0
2,4.856,0.0,1.0,0.0,0.0,92.0,0
3,4.120,0.0,0.0,0.0,0.0,1468.0,1
4,4.963,0.0,0.0,0.0,0.0,36.0,0


Todos os preparativos prontos podemos fazer nosso modelo prever os dados que ainda não viu, porém a forma que nos é devolvida é dificil de visualizarmos, então será necessário fazermos nossa matriz de confusão do mesmo jeito que antes.

In [12]:
y_test = df_val['y']
x_test = df_val[['interest_rate','credit','march','previous','duration']]
validation = results_logit.predict(x_test)
validation = validation.apply(lambda x: 1 if x>=0.5 else 0)
validation = validation.to_frame()
y_test = y_test.to_frame()
validation = validation.merge(y_test, left_on=None, right_on=None, left_index=True, right_index=True)
validation['matrix'] = 0
for idx, value in validation.iterrows():
    if (validation.iloc[idx,0]==1) & (validation.iloc[idx,1]==1):    #True Positive
        validation.iloc[idx,-1] = 0
    elif (validation.iloc[idx,0]==1) & (validation.iloc[idx,1]==0):  #False Positive
        validation.iloc[idx,-1] = 1
    elif (validation.iloc[idx,0]==0) & (validation.iloc[idx,1]==0):  #True Negative
        validation.iloc[idx,-1] = 2
    elif (validation.iloc[idx,0]==0) & (validation.iloc[idx,1]==1):  #False Negative
        validation.iloc[idx,-1] = 3
validation['matrix'].value_counts()

0    98
2    93
1    18
3    13
Name: matrix, dtype: int64

In [13]:
val_conf_mat = pd.DataFrame(validation['matrix'].value_counts())
val_conf_mat = val_conf_mat.reindex([0, 1, 3, 2])
temp = val_conf_mat.iloc[2:]
val_conf_mat.drop([3,2], axis=0,inplace=True)
val_conf_mat['Previsto: Negativo'] = temp.values
val_conf_mat.index = ['Real: Positivo', 'Real: Negativo']
val_conf_mat.columns = ['Previsto: Positivo', 'Previsto: Negativo']
val_conf_mat

,Previsto: Positivo,Previsto: Negativo
Real: Positivo,98,13
Real: Negativo,18,93


Matriz de confusão feita, vamos calcular os parâmetros com os dados que essa tabela nos passa.

In [14]:
val_recall = val_conf_mat.iloc[0,0]/(val_conf_mat.iloc[0,0]+val_conf_mat.iloc[0,1])
val_precisao = val_conf_mat.iloc[0,0]/(val_conf_mat.iloc[0,0]+val_conf_mat.iloc[1,0])
val_acuracia = (val_conf_mat.iloc[0,0]+val_conf_mat.iloc[1,1])/val_conf_mat.sum().sum()
val_fscore = 2*((val_precisao*val_recall)/(val_precisao+val_recall))
print('Recall: {0}'.format(round(val_recall,2)))
print('Precisão: {0}'.format(round(val_precisao,2)))
print('Acurácia: {0}'.format(round(val_acuracia,2)))
print('F-score: {0}'.format(round(val_fscore,2)))

Recall: 0.88
Precisão: 0.84
Acurácia: 0.86
F-score: 0.86


Obtivemos os parâmetros da validação, mas será que eles são maiores ou menores que os parâmetros de teste?<br>
Vamos imprimir os valores lado a lado para compararmos.

In [15]:
print('\t\tTeste \t Validação')
print('Recall: \t{0} \t {1}'.format(round(recall,4),round(val_recall,4)))
print('Precisão: \t{0} \t {1}'.format(round(precisao,4),round(val_precisao,4)))
print('Acurácia: \t{0} \t {1}'.format(round(acuracia,4),round(val_acuracia,4)))
print('F-score: \t{0} \t {1}'.format(round(fscore,4),round(val_fscore,4)))

		Teste 	 Validação
Recall: 	0.8417 	 0.8829
Precisão: 	0.8862 	 0.8448
Acurácia: 	0.8668 	 0.8604
F-score: 	0.8634 	 0.8634


Como podemos ver excluindo o <i>Recall</i> que aumentou e o <i>F-score</i> que manteve o mesmo, o resto dos parâmetros diminuiram, porém era esperado, pois quando estavamos criando a regressão o <i>StatsModels</i> nos retornou o seguinte texto.<br>
<blockquote>
Optimization terminated successfully.<br>
Current function value: 0.336668<br>
Iterations 7
</blockquote>
Isso nos dá que a biblioteca está utilizando machine learning para parametrizar o modelo nos nossos dados, retornando o valor da objetive function da última iteração e o número de iterações necessárias, com isso podemos esperar que haja um pouco de overfitting. Entretando a diminuição da <i>acurácia</i> não é tão grande tanto que o <i>f-score</i> se manteve o mesmo.